In [1]:
import json
import pandas as pd

In [2]:
#How to write dictionary to json file
import json
auth_dict={'consumer_key':'Qlea4Do8JXfjrr6KeeDB2lTT0','consumer_secret':'SaqbpgrIFlKdVAtOrqZVRl6CwKNtHH0Y37qYROP9Tg3Amc4aIC',
           'token':'956618894902513666-eIm7uHFmjAAUzEYHuqNWsWCRx4ex27L','token_secret':'rgwKtFntBckRheF4HUNzdd16gQFdwqV6iLZ4mYTl97Tkl'}
js = json.dumps(auth_dict)
fp = open('auth_dict.json','w')
fp.write(js)
fp.close()

In [3]:
import twitter
from twitter import Twitter
from twitter import OAuth

In [4]:
# Loading my authentication tokens
with open('auth_dict.json','r') as f:
    twtr_auth = json.load(f)

# To make it more readable, lets store
# the OAuth credentials in strings first.
CONSUMER_KEY = twtr_auth['consumer_key']
CONSUMER_SECRET = twtr_auth['consumer_secret']
OAUTH_TOKEN = twtr_auth['token']
OAUTH_TOKEN_SECRET = twtr_auth['token_secret']
    
# Then, we store the OAuth object in "auth"
auth = OAuth(OAUTH_TOKEN, OAUTH_TOKEN_SECRET,
                           CONSUMER_KEY, CONSUMER_SECRET)
# Notice that there are four tokens - you need to create these in the
# Twitter Apps dashboard after you have created your own "app".

# We now create the twitter search object.
t = Twitter(auth=auth)

# The Friendship Graph

In [5]:
import sys
import time
from twitter.api import TwitterHTTPError
from urllib.error import URLError
from http.client import BadStatusLine

def make_twitter_request(twitter_api_func, max_errors=10, *args, **kw):
    # A nested helper function that handles common HTTPErrors. Return an updated
    # value for wait_period if the problem is a 500 level error. Block until the
    # rate limit is reset if it's a rate limiting issue (429 error). Returns None
    # for 401 and 404 errors, which requires special handling by the caller.
    def handle_twitter_http_error(e, wait_period=2, sleep_when_rate_limited=True):
        if wait_period > 3600: # Seconds
            print('Too many retries. Quitting.', file=sys.stderr)
            raise e
        if e.e.code == 401:
            return None
        elif e.e.code == 404:
            print('Encountered 404 Error (Not Found)', file=sys.stderr)
            return None
        elif e.e.code == 429:
            print('Encountered 429 Error (Rate Limit Exceeded)', file=sys.stderr)
            if sleep_when_rate_limited:
                print("Retrying in 15 minutes...ZzZ...", file=sys.stderr)
                sys.stderr.flush()
                time.sleep(60*15 + 5)
                print('...ZzZ...Awake now and trying again.', file=sys.stderr)
                return 2
            else:
                raise e # Caller must handle the rate limiting issue
        elif e.e.code in (500, 502, 503, 504):
            print('Encountered %i Error. Retrying in %i seconds' % (e.e.code, wait_period), file=sys.stderr)
            time.sleep(wait_period)
            wait_period *= 1.5
            return wait_period
        else:
            raise e

    # End of nested helper function

    wait_period = 2
    error_count = 0
    while True:
        try:
            return twitter_api_func(*args, **kw)
        except TwitterHTTPError as e:
            error_count = 0
            wait_period = handle_twitter_http_error(e, wait_period)
            if wait_period is None:
                return
        except URLError as e:
            error_count += 1
            print("URLError encountered. Continuing.", file=sys.stderr)
            if error_count > max_errors:
                print("Too many consecutive errors...bailing out.", file=sys.stderr)
                raise
        except BadStatusLine as e:
            error_count += 1
            print >> sys.stderr, "BadStatusLine encountered. Continuing."
            if error_count > max_errors:
                print("Too many consecutive errors...bailing out.", file=sys.stderr)
                raise

In [6]:
# This will let us create new partial
# functions with arguments set to 
# certain values.
from functools import partial

# This was maxint.
# There is no longer a maxint (in Python 3)
from sys import maxsize


def get_friends_followers_ids(twitter_api, screen_name=None, user_id=None,
                                friends_limit=maxsize, followers_limit=maxsize):
    # Must have either screen_name or user_id (logical xor)
    assert (screen_name != None) != (user_id != None), \
    "Must have screen_name or user_id, but not both"
    
    # You can also do this with a function closure.
    get_friends_ids = partial(make_twitter_request, twitter_api.friends.ids,
                                count=5000)
    get_followers_ids = partial(make_twitter_request, twitter_api.followers.ids,
                                count=5000)
    friends_ids, followers_ids = [], []
    for twitter_api_func, limit, ids, label in [
            [get_friends_ids, friends_limit, friends_ids, "friends"],
            [get_followers_ids, followers_limit, followers_ids, "followers"]
            ]:
        #LOOK HERE! This little line is important.
        if limit == 0: continue
        cursor = -1
        while cursor != 0:
            # Use make_twitter_request via the partially bound callable...
            if screen_name:
                response = twitter_api_func(screen_name=screen_name, cursor=cursor)
            else: # user_id
                response = twitter_api_func(user_id=user_id, cursor=cursor)
            if response is not None:
                ids += response['ids']
                cursor = response['next_cursor']
            print('Fetched {0} total {1} ids for {2}'.format(len(ids),
                    label, (user_id or screen_name), file=sys.stderr))
            if len(ids) >= limit or response is None:
                break
    # Do something useful with the IDs, like store them to disk...
    return friends_ids[:friends_limit], followers_ids[:followers_limit]



In [7]:
friends_ids, followers_ids = get_friends_followers_ids(t,
                                screen_name="ZedShaw",
                                friends_limit=10,
                                followers_limit=10)
print(friends_ids)
print(followers_ids)

Fetched 988 total friends ids for ZedShaw
Fetched 5000 total followers ids for ZedShaw
[768486347300626432, 22455722, 16948493, 402502573, 2557037323, 3065618342, 50393960, 111778335, 787947846, 101798589]
[2331267277, 14484731, 750964547868033024, 2335654568, 14509720, 2323455785, 267032476, 3671264597, 37025099, 939122210081456128]


In [8]:
!pip install tweepy

In [9]:
import tweepy
consumer_key = 'Qlea4Do8JXfjrr6KeeDB2lTT0'
consumer_secret = 'SaqbpgrIFlKdVAtOrqZVRl6CwKNtHH0Y37qYROP9Tg3Amc4aIC'
access_token = '956618894902513666-eIm7uHFmjAAUzEYHuqNWsWCRx4ex27L'
access_token_secret = 'rgwKtFntBckRheF4HUNzdd16gQFdwqV6iLZ4mYTl97Tkl'
auth = tweepy.OAuthHandler(consumer_key, consumer_secret)
auth.set_access_token(access_token, access_token_secret)
api = tweepy.API(auth)
user = api.get_user(screen_name = 'ZedShaw') 
user.id


15029296

In [10]:
# Create a mostly empty data frame,
# and write it to a CSV file.
df = pd.DataFrame(columns=['ID','followers'])
df.to_csv('followers.csv', index=False)

# Our function
def save_followers(fid, followers):
    df = pd.DataFrame([[fid, followers]], columns=['ID','followers'])
    with open('followers.csv', 'a') as f:
        df.to_csv(f,header=False, index=False)

In [11]:
# Selecting a user
screen_name = 'ZedShaw'

# Getting friends' ids
response = make_twitter_request(t.friends.ids,
                                screen_name=screen_name, count = 5000)
friends = response["ids"]

# Getting followers' ids
response = make_twitter_request(t.followers.ids,
                                screen_name=screen_name, count = 5000)
followers = response["ids"]

# Computing reciprocal friends
reciprocal_friends = set(friends) & set(followers)

In [12]:
response

{'ids': [2331267277,
  14484731,
  750964547868033024,
  2335654568,
  14509720,
  2323455785,
  267032476,
  3671264597,
  37025099,
  939122210081456128,
  36059443,
  751193930926989313,
  1416799622,
  20129803,
  371288198,
  311358522,
  469816534,
  1108286720,
  17026194,
  829918520,
  1454590098,
  2707932437,
  7069202,
  20395694,
  824806994594889729,
  955138259885096960,
  539596280,
  118063398,
  354810026,
  921679708605108224,
  37290791,
  934439361684307968,
  864083118986317824,
  145969873,
  317867812,
  807282420143816704,
  2055351,
  634245122,
  342512565,
  26068518,
  159351385,
  2519550648,
  107456998,
  1525891243,
  22861497,
  7382162,
  952725835919904768,
  965414996866777088,
  844304603336232960,
  103693215,
  1230096818,
  6880942,
  15493467,
  713085430615420928,
  7943972,
  3229054291,
  739988339869122561,
  19662502,
  269972981,
  16250884,
  10455532,
  17006808,
  79126199,
  956193530921308161,
  31129627,
  14138576,
  791857,
  9664

In [13]:
type(response)

twitter.api.TwitterDictResponse

In [14]:
reciprocal_friends

{41573,
 355203,
 771681,
 982721,
 1020961,
 1434051,
 3286561,
 3562421,
 5743852,
 5800582,
 7381872,
 7579442,
 8504952,
 8513352,
 9929452,
 10179552,
 10187892,
 12735452,
 13138322,
 14030452,
 14076724,
 14472613,
 14582359,
 14759016,
 15740084,
 15759497,
 15808076,
 16009329,
 16147002,
 16788716,
 17198807,
 17876157,
 20522624,
 22386062,
 22552500,
 26816492,
 27594274,
 30072091,
 34877300,
 77827772,
 85024364,
 100838242,
 130844081,
 137159973,
 143360147,
 149730311,
 150348736,
 153966123,
 154639065,
 156711501,
 165478097,
 177788470,
 187354174,
 229311487,
 291521729,
 306544720,
 319769408,
 326511843,
 347488218,
 388958428,
 454915914,
 602869845,
 1023779768,
 1500013435,
 2409545137,
 2693104332,
 2904757123,
 3115325280,
 3231781692,
 3562121415,
 4129733188,
 4778220615,
 4919024180,
 709828044169547777,
 823412906859622400}

In [15]:
def get_user_profile(twitter_api, screen_names=None, user_ids=None):
    # Must have either screen_name or user_id (logical xor)
    assert (screen_names != None) != (user_ids != None) 
    #"Must have screen_names or user_ids, but not both"
    items_to_info = {}
    items = screen_names or user_ids 
    while len(items) > 0:
        # Process 100 items at a time per the API specifications for /users/lookup.
        # See https://dev.twitter.com/docs/api/1.1/get/users/lookup for details.
        items_str = ','.join([str(item) for item in items[:100]]) 
        items = items[100:]
        if screen_names:
            response = make_twitter_request(twitter_api.users.lookup,screen_name=items_str)
        else:
            response = make_twitter_request(twitter_api.users.lookup,user_id=items_str)
        for user_info in response: 
            if screen_names:
                items_to_info[user_info['screen_name']] = user_info 
            else: # user_ids
                items_to_info[user_info['id']] = user_info 
    return items_to_info
# Sample usage
#twitter_api = oauth_login()
#print get_user_profile(twitter_api, screen_names=["SocialWebMining", "ptwobrussell"])
#print get_user_profile(twitter_api, user_ids=[132373965])

In [16]:
follow_count=get_user_profile(t,user_ids=[reciprocal_friends])

In [17]:
follow_count

{41573: {'contributors_enabled': False,
  'created_at': 'Mon Dec 04 23:35:51 +0000 2006',
  'default_profile': False,
  'default_profile_image': False,
  'description': 'Computers, Cameras, Space, 🐢🐢🐢🐢🐀🍕  https://t.co/YRhi8wqbBR',
  'entities': {'description': {'urls': [{'display_url': 'treypiepmeier.com',
      'expanded_url': 'http://treypiepmeier.com',
      'indices': [35, 58],
      'url': 'https://t.co/YRhi8wqbBR'}]},
   'url': {'urls': [{'display_url': 'giving.cofc.edu/piepmeier',
      'expanded_url': 'https://giving.cofc.edu/piepmeier',
      'indices': [0, 23],
      'url': 'https://t.co/jMiApJ2GJl'}]}},
  'favourites_count': 24462,
  'follow_request_sent': False,
  'followers_count': 1798,
  'following': False,
  'friends_count': 338,
  'geo_enabled': True,
  'has_extended_profile': True,
  'id': 41573,
  'id_str': '41573',
  'is_translation_enabled': False,
  'is_translator': False,
  'lang': 'en',
  'listed_count': 122,
  'location': 'New Bedford, MA / inside my own heart'

In [18]:
type(follow_count)

dict

In [19]:
follow_count.keys()

dict_keys([709828044169547777, 823412906859622400, 2904757123, 355203, 5800582, 149730311, 15759497, 22386062, 13138322, 143360147, 30072091, 1020961, 27594274, 3286561, 137159973, 14472613, 153966123, 130844081, 2409545137, 7579442, 4919024180, 14076724, 3562421, 15740084, 1023779768, 177788470, 16147002, 22552500, 3231781692, 17876157, 187354174, 77827772, 319769408, 150348736, 291521729, 1434051, 4129733188, 982721, 3562121415, 4778220615, 8513352, 454915914, 2693104332, 156711501, 15808076, 306544720, 165478097, 602869845, 14582359, 17198807, 154639065, 347488218, 388958428, 12735452, 10179552, 771681, 100838242, 326511843, 3115325280, 41573, 14759016, 9929452, 16788716, 5743852, 85024364, 7381872, 16009329, 26816492, 10187892, 14030452, 34877300, 8504952, 1500013435])

In [20]:
follow_count[771681]['followers_count']

10572

In [21]:
new_follow_count=list(sorted(follow_count, key=lambda x: (follow_count[x]['followers_count']),reverse=True))[:5]

In [22]:
new_follow_count

[153966123, 5800582, 1500013435, 319769408, 3562121415]

In [26]:
list1=new_follow_count
cnt=0

In [27]:
def rnsrishi(ID,cnt):
    while cnt<20:
        
        friends_ids, followers_ids = get_friends_followers_ids(t,
                                user_id=ID,
                                friends_limit=10,
                                followers_limit=10)
    #print(friends_ids)
    #print(followers_ids)
    #15029296
    # Selecting a user
    #screen_name = 'ZedShaw'
    #user_id=ID

    # Getting friends' ids
        response = make_twitter_request(t.friends.ids,user_id=ID, count = 5000)
        #friends =[]
        friends = response["ids"]

    # Getting followers' ids
        response = make_twitter_request(t.followers.ids,user_id=ID, count = 5000)
        #followers = []
        followers = response["ids"]

    # Computing reciprocal friends
        reciprocal_friends = set(friends) & set(followers)

    #reciprocal_friends

        follow_count=get_user_profile(t,user_ids=[reciprocal_friends])

        new_follow_count=list(sorted(follow_count, key=lambda x: (follow_count[x]['followers_count']),reverse=True))[:5]
    
        new_follow_count

        for n in new_follow_count:
            list1.append(n)
        
        ID=list1[cnt]
        cnt += 1
        #rnsrishi(ID,cnt)
    
        #list1

In [28]:
rnsrishi(15029296,cnt)

Fetched 988 total friends ids for 15029296
Fetched 5000 total followers ids for 15029296
Fetched 5000 total friends ids for 153966123
Fetched 5000 total followers ids for 153966123


TwitterHTTPError: Twitter sent status 403 for URL: 1.1/users/lookup.json using parameters: (oauth_consumer_key=Qlea4Do8JXfjrr6KeeDB2lTT0&oauth_nonce=16809195699594038987&oauth_signature_method=HMAC-SHA1&oauth_timestamp=1519592226&oauth_token=956618894902513666-eIm7uHFmjAAUzEYHuqNWsWCRx4ex27L&oauth_version=1.0&user_id=%7B795696778381299712%2C%20872863353927901184%2C%20828660421477404674%2C%20823786833746067458%2C%20780987094734864386%2C%20832785993208066050%2C%20833000805884129284%2C%20872103010033438720%2C%20874082097299718144%2C%20851854036953202688%2C%20822043861778317312%2C%203081916939%2C%20442851852%2C%201928030214%2C%20724694790047170561%2C%20895991824493445121%2C%20626249746%2C%202829267474%2C%20424759316%2C%202159782935%2C%201071442969%2C%203232299033%2C%202840388132%2C%2033829414%2C%20749691433%2C%20890290834985418753%2C%2021121069%2C%20217458224%2C%203333457456%2C%204362867257%2C%204220203066%2C%20331097662%2C%20373882371%2C%20730995277%2C%202537981006%2C%203245238868%2C%2054

In [ ]:
list1

In [110]:
len(list1)

14

In [104]:
new_follow_count

[153966123, 5800582, 1500013435, 319769408, 3562121415]

In [105]:
cnt

1